### 60. KVSの構築
Key-Value-Store (KVS) を用い，アーティスト名（name）から活動場所（area）を検索するためのデータベースを構築せよ．

In [9]:
import leveldb
db60 = leveldb.LevelDB('data/db60')

In [1]:
import gzip
import json

In [10]:
for line in gzip.open('data/artist.json.gz'):
    data  = json.loads(line)
    name = (data['name'] + ' ' + str(data['id'])).encode()
    area = data.get('area', 'NODATA').encode()
    db60.Put(name, area)

### 61. KVSの検索
60で構築したデータベースを用い，特定の（指定された）アーティストの活動場所を取得せよ．

In [4]:
def where(name):
    for k, v in db60.RangeIter(key_from=name.encode(), key_to=(name + chr(33)).encode()):
        artist = k.decode().split()[0]
        place = v.decode()
        print(artist + '\t' + place)

In [5]:
where('Oasis')

Oasis	United Kingdom
Oasis	United States
Oasis	United Kingdom
Oasis	NODATA
Oasis	NODATA
Oasis	Kent


### 62. KVS内の反復処理
60で構築したデータベースを用い，活動場所が「Japan」となっているアーティスト数を求めよ．

In [6]:
i = 0
for k, v in db60.RangeIter():
    if v.decode() == 'Japan':
        i = i + 1
print(i)

22821


### 63. オブジェクトを値に格納したKVS
KVSを用い，アーティスト名（name）からタグと被タグ数（タグ付けされた回数）のリストを検索するためのデータベースを構築せよ．さらに，ここで構築したデータベースを用い，アーティスト名からタグと被タグ数を検索せよ．

In [7]:
db63 = leveldb.LevelDB('data/db63')

In [8]:
for line in gzip.open('data/artist.json.gz'):
    data  = json.loads(line)
    name = (data['name'] + ' ' + str(data['id'])).encode()
    if 'tags' in data:
        tags = ' '.join([tag['value'] + ' ' + str(tag['count']) for tag in data['tags']]).encode()
    else:
        tags = 'NOTAGS'.encode()
    db63.Put(name, tags)

In [9]:
def taged(name):
    for k, v in db63.RangeIter(key_from=name.encode(), key_to=(name + chr(33)).encode()):
        artist = k.decode().split()[0]
        tags = v.decode()
        print(artist + '\t' + tags)

In [10]:
taged('Oasis')

Oasis	rock 1 britpop 3 british 4 uk 1 britannique 1 rock and indie 1 england 1 manchester 1
Oasis	NOTAGS
Oasis	morning glory 1 oasis 1
Oasis	NOTAGS
Oasis	NOTAGS
Oasis	NOTAGS


### 64. MongoDBの構築
アーティスト情報（artist.json.gz）をデータベースに登録せよ．さらに，次のフィールドでインデックスを作成せよ: name, aliases.name, tags.value, rating.value

In [2]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)

In [3]:
db = client['database64']

In [4]:
collection = db['collection64']

In [6]:
for line in gzip.open('data/artist.json.gz'):
    data  = json.loads(line)
    collection.insert_one(data)

In [8]:
collection.count_documents({})

921337

In [20]:
sorted(list(collection.index_information()))[:10]

['_id_', 'name_1']

In [19]:
collection.create_index('name')

'name_1'

In [45]:
collection.create_index('aliases.name')

'aliases.name_1'

In [48]:
collection.create_index('tags.value')

'tags.value_1'

In [61]:
collection.create_index('rating.value')

'rating.value_1'

### 65. MongoDBの検索
MongoDBのインタラクティブシェルを用いて，"Queen"というアーティストに関する情報を取得せよ．さらに，これと同様の処理を行うプログラムを実装せよ．

In [59]:
i=0
for line in gzip.open('data/artist.json.gz'):
    data  = json.loads(line)
    print(type(data.get('rating',{}).get('value')))
    #print([d['value'] for d in data.get('rating',[])])
    i=i+1
    if i == 100:
        break

<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'int'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'

In [ ]:
i = 0
for post in collection.find_one()

In [43]:
for post in collection.find({'name':'Oasis'}):
    print(post)

{'_id': ObjectId('5cb96a1582efee225266a654'), 'name': 'Oasis', 'area': 'United Kingdom', 'tags': [{'count': 1, 'value': 'morning glory'}, {'count': 1, 'value': 'oasis'}], 'sort_name': 'Oasis', 'ended': True, 'gid': 'a7a848c3-515d-4486-9e69-dc7491dd3bd1', 'type': 'Group', 'id': 377879}
{'_id': ObjectId('5cb96af782efee22526c1947'), 'name': 'Oasis', 'area': 'United States', 'sort_name': 'Oasis', 'ended': True, 'gid': 'ecf9f3a3-35e9-4c58-acaa-e707fba45060', 'type': 'Group', 'id': 286198}
{'_id': ObjectId('5cb96b0e82efee22526ca96a'), 'rating': {'count': 13, 'value': 86}, 'begin': {'year': 1991}, 'end': {'date': 28, 'month': 8, 'year': 2009}, 'name': 'Oasis', 'area': 'United Kingdom', 'tags': [{'count': 1, 'value': 'rock'}, {'count': 3, 'value': 'britpop'}, {'count': 4, 'value': 'british'}, {'count': 1, 'value': 'uk'}, {'count': 1, 'value': 'britannique'}, {'count': 1, 'value': 'rock and indie'}, {'count': 1, 'value': 'england'}, {'count': 1, 'value': 'manchester'}], 'sort_name': 'Oasis', 'e